# 모듈 import

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
pd.set_option('display.max_columns', None) # 데이터프레임 컬럼 display

import warnings
warnings.filterwarnings('ignore') # 경고 무시

In [2]:
from selenium import webdriver
# 크롬 브라우저
driver = webdriver.Chrome('C:/chromedrive/chromedriver.exe')
# browser = webdriver.Chrome('C:/chromedrive/chromedriver.exe')

<br/>
<br/>
<br/>
<br/>

In [116]:
df['세트_성공률'] = round(df['세트_성공']/df['세트_시도']*100, 2)
df['블로킹_성공률'] = round(df['블로킹_성공']/df['블로킹_시도']*100, 2)
df['디그_성공률'] = round(df['디그_성공']/df['디그_시도']*100, 2)

# 세트성공률, 디그 성공률, 블로킹, 퀵오픈공격 성공률
df_v2 = df.loc[:, ["팀명", "경기날짜", "결과", '공격종합_성공률','서브_성공률',"세트_성공률", "디그_성공률", "블로킹_성공률", "퀵오픈_성공률",'득점점유율','범실_범실','포지션','스타팅멤버']]
df_v2.fillna(0, inplace=True)
df_v2.isna().sum()

팀명          0
경기날짜        0
결과          0
공격종합_성공률    0
서브_성공률      0
세트_성공률      0
디그_성공률      0
블로킹_성공률     0
퀵오픈_성공률     0
득점점유율       0
범실_범실       0
포지션         0
스타팅멤버       0
dtype: int64

In [118]:
df_v2

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션,스타팅멤버
0,IBK기업은행,2017-10-14,0,29.85,0.20,0.00,83.33,15.00,45.45,0.347826,7,L,1
1,IBK기업은행,2017-10-14,0,30.00,0.60,14.29,77.27,0.00,60.00,0.217391,6,L,1
2,IBK기업은행,2017-10-14,0,27.78,0.20,0.00,91.67,28.57,50.00,0.217391,6,R,1
3,IBK기업은행,2017-10-14,0,31.25,0.00,0.00,83.33,16.67,60.00,0.101449,1,L,1
4,IBK기업은행,2017-10-14,0,25.00,0.20,0.00,90.91,3.57,0.00,0.086957,3,C,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8111,KGC인삼공사,2022-02-28,0,28.57,0.00,0.00,0.00,0.00,0.00,0.088889,0,C,1
8112,KGC인삼공사,2022-02-28,0,60.00,0.00,0.00,0.00,0.00,0.00,0.066667,0,C,1
8113,KGC인삼공사,2022-02-28,0,0.00,0.33,0.00,0.00,0.00,0.00,0.022222,1,L,0
8114,KGC인삼공사,2022-02-28,0,0.00,0.33,0.00,0.00,0.00,0.00,0.022222,0,S,0


<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# 머신러닝
- 데이터 변경
- 데이터 분리
- 모델링

#### 데이터 변경
- A, B팀의 스타팅 멤버들의 직전 경기력으로 승패 예측

In [119]:
# 한 경기당 스타팅멤버의 수 확인
df_v2.groupby(['경기날짜', '팀명'])['스타팅멤버'].sum().value_counts()

6    673
5    178
4     61
3     21
7     10
2      5
Name: 스타팅멤버, dtype: int64

In [120]:
# 스타팅멤버만 선택
df_starting = df_v2[df_v2['스타팅멤버'] == 1]
df_starting

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션,스타팅멤버
0,IBK기업은행,2017-10-14,0,29.85,0.2,0.00,83.33,15.00,45.45,0.347826,7,L,1
1,IBK기업은행,2017-10-14,0,30.00,0.6,14.29,77.27,0.00,60.00,0.217391,6,L,1
2,IBK기업은행,2017-10-14,0,27.78,0.2,0.00,91.67,28.57,50.00,0.217391,6,R,1
3,IBK기업은행,2017-10-14,0,31.25,0.0,0.00,83.33,16.67,60.00,0.101449,1,L,1
4,IBK기업은행,2017-10-14,0,25.00,0.2,0.00,90.91,3.57,0.00,0.086957,3,C,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8108,KGC인삼공사,2022-02-28,0,43.90,0.0,0.00,0.00,0.00,50.00,0.466667,2,R,1
8109,KGC인삼공사,2022-02-28,0,28.57,0.0,0.00,0.00,0.00,50.00,0.200000,2,L,1
8110,KGC인삼공사,2022-02-28,0,30.00,0.0,0.00,0.00,0.00,33.33,0.133333,4,L,1
8111,KGC인삼공사,2022-02-28,0,28.57,0.0,0.00,0.00,0.00,0.00,0.088889,0,C,1


In [121]:
## 경기 당 스타팅멤버가 5명 이상만 선택

# 스타팅멤버가 5명 미만이 포함된 날짜 선택
temp = (df_starting.groupby(['경기날짜', '팀명'])[['스타팅멤버']].sum() < 5).reset_index()
drop_dates = temp.loc[temp['스타팅멤버'], '경기날짜'].unique()

# drop_dates를 제외한 날짜만 선택
df_starting = df_starting[~df_starting['경기날짜'].isin(drop_dates)]

# 스타팅멤버 다시 확인
df_starting.groupby(['경기날짜', '팀명'])['스타팅멤버'].sum().value_counts()

6    613
5    151
7     10
Name: 스타팅멤버, dtype: int64

In [122]:
df_starting.drop(['스타팅멤버'],axis=1,inplace=True)

In [123]:
df_starting

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션
0,IBK기업은행,2017-10-14,0,29.85,0.2,0.00,83.33,15.00,45.45,0.347826,7,L
1,IBK기업은행,2017-10-14,0,30.00,0.6,14.29,77.27,0.00,60.00,0.217391,6,L
2,IBK기업은행,2017-10-14,0,27.78,0.2,0.00,91.67,28.57,50.00,0.217391,6,R
3,IBK기업은행,2017-10-14,0,31.25,0.0,0.00,83.33,16.67,60.00,0.101449,1,L
4,IBK기업은행,2017-10-14,0,25.00,0.2,0.00,90.91,3.57,0.00,0.086957,3,C
...,...,...,...,...,...,...,...,...,...,...,...,...
8108,KGC인삼공사,2022-02-28,0,43.90,0.0,0.00,0.00,0.00,50.00,0.466667,2,R
8109,KGC인삼공사,2022-02-28,0,28.57,0.0,0.00,0.00,0.00,50.00,0.200000,2,L
8110,KGC인삼공사,2022-02-28,0,30.00,0.0,0.00,0.00,0.00,33.33,0.133333,4,L
8111,KGC인삼공사,2022-02-28,0,28.57,0.0,0.00,0.00,0.00,0.00,0.088889,0,C


######  IBK기업은행, 흥국생명, 한국도로공사, 현대건설, GS칼텍스, KGC인삼공사, 페퍼저축은행

In [124]:
IBK기업은행 = df_starting.loc[df_starting['팀명']=='IBK기업은행']
IBK기업은행 = pd.get_dummies(IBK기업은행,columns=['포지션'])
IBK기업은행

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션_C,포지션_L,포지션_R,포지션_S
0,IBK기업은행,2017-10-14,0,29.85,0.20,0.00,83.33,15.00,45.45,0.347826,7,0,1,0,0
1,IBK기업은행,2017-10-14,0,30.00,0.60,14.29,77.27,0.00,60.00,0.217391,6,0,1,0,0
2,IBK기업은행,2017-10-14,0,27.78,0.20,0.00,91.67,28.57,50.00,0.217391,6,0,0,1,0
3,IBK기업은행,2017-10-14,0,31.25,0.00,0.00,83.33,16.67,60.00,0.101449,1,0,1,0,0
4,IBK기업은행,2017-10-14,0,25.00,0.20,0.00,90.91,3.57,0.00,0.086957,3,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8072,IBK기업은행,2022-02-26,1,51.43,0.00,0.00,0.00,0.00,76.47,0.327586,4,1,0,0,0
8073,IBK기업은행,2022-02-26,1,37.50,0.33,0.00,0.00,0.00,40.00,0.172414,4,0,1,0,0
8074,IBK기업은행,2022-02-26,1,0.00,0.33,0.00,0.00,0.00,0.00,0.086207,1,1,0,0,0
8075,IBK기업은행,2022-02-26,1,50.00,0.00,0.00,0.00,0.00,0.00,0.017241,2,1,0,0,0


In [125]:
흥국생명 = df_starting.loc[df_starting['팀명']=='흥국생명']
흥국생명 = pd.get_dummies(흥국생명,columns=['포지션'])
흥국생명

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션_C,포지션_L,포지션_R,포지션_S
8,흥국생명,2017-10-14,1,41.90,0.0,16.67,73.68,30.77,33.33,0.585366,13,0,1,0,0
9,흥국생명,2017-10-14,1,24.39,0.2,0.00,81.48,33.33,100.00,0.158537,3,0,1,0,0
10,흥국생명,2017-10-14,1,30.00,0.2,33.33,75.00,20.00,0.00,0.097561,3,0,0,1,0
12,흥국생명,2017-10-14,1,18.18,0.0,0.00,60.00,7.69,0.00,0.036585,1,1,0,0,0
13,흥국생명,2017-10-14,1,0.00,0.0,33.33,94.74,23.08,0.00,0.036585,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8079,흥국생명,2022-02-26,0,26.53,0.0,0.00,0.00,0.00,13.33,0.351351,2,0,0,1,0
8080,흥국생명,2022-02-26,0,50.00,1.0,0.00,0.00,0.00,100.00,0.243243,2,0,1,0,0
8082,흥국생명,2022-02-26,0,21.43,0.0,0.00,0.00,0.00,0.00,0.108108,2,1,0,0,0
8083,흥국생명,2022-02-26,0,33.33,0.0,0.00,0.00,0.00,0.00,0.081081,1,1,0,0,0


In [126]:
한국도로공사 = df_starting.loc[df_starting['팀명']=='한국도로공사']
한국도로공사 = pd.get_dummies(한국도로공사,columns=['포지션'])
한국도로공사

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션_C,포지션_L,포지션_R,포지션_S
37,한국도로공사,2017-10-17,0,50.00,0.60,16.67,60.00,28.57,40.00,0.341463,9,0,0,1,0
38,한국도로공사,2017-10-17,0,34.69,0.00,0.00,100.00,0.00,36.84,0.207317,1,0,1,0,0
39,한국도로공사,2017-10-17,0,45.45,0.20,50.00,88.89,18.75,0.00,0.170732,3,1,0,0,0
40,한국도로공사,2017-10-17,0,42.86,0.00,16.67,100.00,15.79,0.00,0.146341,3,1,0,0,0
41,한국도로공사,2017-10-17,0,35.71,0.40,16.67,90.91,6.67,20.00,0.097561,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7998,한국도로공사,2022-02-21,1,30.95,0.00,0.00,0.00,0.00,38.10,0.186667,2,0,1,0,0
7999,한국도로공사,2022-02-21,1,47.83,0.00,0.00,0.00,0.00,100.00,0.186667,2,1,0,0,0
8000,한국도로공사,2022-02-21,1,25.00,0.25,0.00,0.00,0.00,0.00,0.066667,2,1,0,0,0
8001,한국도로공사,2022-02-21,1,11.11,0.25,0.00,0.00,0.00,25.00,0.053333,4,0,0,1,0


In [127]:
현대건설 = df_starting.loc[df_starting['팀명']=='현대건설']
현대건설 = pd.get_dummies(현대건설,columns=['포지션'])
현대건설

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션_C,포지션_L,포지션_R,포지션_S
28,현대건설,2017-10-15,1,41.46,0.2,0.00,86.36,15.00,40.00,0.372549,8,0,1,0,0
29,현대건설,2017-10-15,1,44.19,0.2,0.00,94.44,12.50,53.85,0.205882,5,0,0,1,0
30,현대건설,2017-10-15,1,43.48,0.4,0.00,100.00,14.29,0.00,0.156863,5,1,0,0,0
31,현대건설,2017-10-15,1,30.77,0.0,50.00,100.00,33.33,0.00,0.117647,1,1,0,0,0
32,현대건설,2017-10-15,1,27.03,0.0,16.67,86.96,9.09,27.27,0.107843,6,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,현대건설,2022-02-25,0,27.27,0.0,0.00,0.00,0.00,27.27,0.114286,6,0,1,0,0
8063,현대건설,2022-02-25,0,46.15,0.0,0.00,0.00,0.00,0.00,0.114286,2,1,0,0,0
8064,현대건설,2022-02-25,0,27.78,0.4,0.00,0.00,0.00,20.00,0.100000,2,0,1,0,0
8065,현대건설,2022-02-25,0,23.53,0.0,0.00,0.00,0.00,22.22,0.071429,2,0,1,0,0


In [128]:
GS칼텍스 = df_starting.loc[df_starting['팀명']=='GS칼텍스']
GS칼텍스 = pd.get_dummies(GS칼텍스,columns=['포지션'])
GS칼텍스

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션_C,포지션_L,포지션_R,포지션_S
47,GS칼텍스,2017-10-17,1,44.68,0.60,50.00,85.71,0.00,52.38,0.272727,7,0,1,0,0
48,GS칼텍스,2017-10-17,1,35.29,0.00,50.00,85.71,25.00,18.18,0.227273,6,0,0,1,0
49,GS칼텍스,2017-10-17,1,37.78,0.40,33.33,94.44,0.00,45.00,0.215909,2,0,0,1,0
50,GS칼텍스,2017-10-17,1,50.00,0.40,0.00,50.00,14.29,0.00,0.125000,1,1,0,0,0
51,GS칼텍스,2017-10-17,1,35.71,0.20,0.00,80.00,0.00,0.00,0.068182,2,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8101,GS칼텍스,2022-02-28,1,55.00,0.33,0.00,0.00,0.00,53.85,0.184615,1,0,1,0,0
8102,GS칼텍스,2022-02-28,1,64.29,0.00,0.00,0.00,0.00,63.64,0.169231,1,0,1,0,0
8103,GS칼텍스,2022-02-28,1,66.67,0.00,0.00,0.00,0.00,0.00,0.123077,0,1,0,0,0
8104,GS칼텍스,2022-02-28,1,66.67,0.67,0.00,0.00,0.00,0.00,0.092308,0,0,0,0,1


In [129]:
KGC인삼공사 = df_starting.loc[df_starting['팀명']=='KGC인삼공사']
KGC인삼공사 = pd.get_dummies(KGC인삼공사,columns=['포지션'])
KGC인삼공사

,팀명,경기날짜,결과,공격종합_성공률,서브_성공률,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,득점점유율,범실_범실,포지션_C,포지션_L,포지션_R,포지션_S
19,KGC인삼공사,2017-10-15,0,38.89,0.0,9.09,76.92,11.11,50.00,0.564103,5,0,0,1,0
20,KGC인삼공사,2017-10-15,0,32.35,0.0,40.00,100.00,20.00,33.33,0.166667,1,0,1,0,0
21,KGC인삼공사,2017-10-15,0,28.57,0.2,33.33,100.00,18.18,0.00,0.064103,1,1,0,0,0
22,KGC인삼공사,2017-10-15,0,13.79,0.0,40.00,75.00,0.00,25.00,0.051282,3,0,1,0,0
23,KGC인삼공사,2017-10-15,0,40.00,0.4,32.69,83.33,0.00,0.00,0.051282,2,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8108,KGC인삼공사,2022-02-28,0,43.90,0.0,0.00,0.00,0.00,50.00,0.466667,2,0,0,1,0
8109,KGC인삼공사,2022-02-28,0,28.57,0.0,0.00,0.00,0.00,50.00,0.200000,2,0,1,0,0
8110,KGC인삼공사,2022-02-28,0,30.00,0.0,0.00,0.00,0.00,33.33,0.133333,4,0,1,0,0
8111,KGC인삼공사,2022-02-28,0,28.57,0.0,0.00,0.00,0.00,0.00,0.088889,0,1,0,0,0


In [154]:
페퍼저축은행 = df_starting.loc[df_starting['팀명']=='페퍼저축은행']
페퍼저축은행 = pd.get_dummies(페퍼저축은행,columns=['포지션'])
페퍼저축은행.shape

(119, 15)

In [158]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
X = IBK기업은행.iloc[:,3:]
y = IBK기업은행.iloc[:,2]
x_tr, x_te, y_tr, y_te = train_test_split(X,y,
                                         random_state=0,
                                        stratify=y
                                    )

knn10 = KNeighborsClassifier(n_neighbors=10)
knn5 = KNeighborsClassifier(n_neighbors=5)
lr = LogisticRegression(max_iter=1000)
dt5 = DecisionTreeClassifier(max_depth=5)
dt7 = DecisionTreeClassifier(max_depth=7)

In [160]:
from sklearn.ensemble import VotingClassifier
hard = VotingClassifier([('knn10',knn10),('knn5',knn5),('lr',lr),
                        ('dt5',dt5),('dt7',dt7)])
soft = VotingClassifier([('knn10',knn10),('knn5',knn5),('lr',lr),
                        ('dt5',dt5),('dt7',dt7)],voting='soft')

In [163]:
names = ['hard','soft','knn10','knn5','lr','dt5','dt7']

for idx, model in enumerate([hard,soft,knn10,lr,dt5,dt7]) :
    model.fit(x_tr,y_tr)
    name = names[idx]
    train_score = model.score(x_tr,y_tr) * 100
    test_score = model.score(x_te,y_te) * 100
    print(f'{name} Train Accuracy:{train_score:.2f}%')
    print(f'{name} Test Accuracy:{test_score:.2f}%')    
    print()

hard Train Accuracy:77.00%
hard Test Accuracy:57.81%

soft Train Accuracy:84.49%
soft Test Accuracy:57.81%

knn10 Train Accuracy:65.51%
knn10 Test Accuracy:57.81%

knn5 Train Accuracy:62.37%
knn5 Test Accuracy:60.94%

lr Train Accuracy:74.04%
lr Test Accuracy:56.25%

dt5 Train Accuracy:84.84%
dt5 Test Accuracy:52.60%



In [164]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5).fit(x_tr,y_tr)
model.score(x_tr,y_tr), model.score(x_te,y_te)

(0.7996515679442509, 0.6041666666666666)

In [165]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier().fit(x_tr,y_tr)
model.score(x_tr,y_tr), model.score(x_te,y_te)

(0.9024390243902439, 0.640625)

In [169]:
from sklearn.ensemble import StackingClassifier
estimators = [('rf',RandomForestClassifier()),('gb',GradientBoostingClassifier())]
model = StackingClassifier(estimators=estimators,final_estimator=LogisticRegression())
model.fit(x_tr,y_tr).score(x_te,y_te)

0.6145833333333334

In [176]:
import lightgbm as lgb
from sklearn.metrics import r2_score

# model = xgb.XGBRegressor(objective='reg:binary').fit(x_tr,y_tr)

lgb_tr = lgb.Dataset(x_tr,y_tr)
lgb_eval = lgb.Dataset(x_te,y_te,reference=lgb_tr)

params = {
    'objective' : 'regression',
}
model = lgb.train(params,lgb_tr, valid_sets=lgb_eval,
                 early_stopping_rounds=5)

p_train = model.predict(x_tr,num_iteration=model.best_iteration)
p_test = model.predict(x_te,num_iteration=model.best_iteration)

r2_score(y_tr,p_train), r2_score(y_te,p_test)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 475
[LightGBM] [Info] Number of data points in the train set: 574, number of used features: 12
[LightGBM] [Info] Start training from score 0.454704
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's l2: 0.245045
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's l2: 0.243647
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's l2: 0.239924
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's l2: 0.23947
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's l2: 0.237678
[LightGBM] [Warning] No f

(0.45657085150548504, 0.07766677265627053)

In [1]:
def pre(x) :
    X = x.iloc[:,3:]
    y = x.iloc[:,2]
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.linear_model import LinearRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.svm import SVC
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import train_test_split


    x_tr, x_te, y_tr, y_te = train_test_split(X,y,
                                         random_state=0,
                                        stratify=y
                                    )
    
    knn10 = KNeighborsClassifier(n_neighbors=10)
    knn5 = KNeighborsClassifier(n_neighbors=5)
    lr = LogisticRegression(max_iter=1000)
    dt5 = DecisionTreeClassifier(max_depth=5)
    dt7 = DecisionTreeClassifier(max_depth=7)
    
    from sklearn.ensemble import VotingClassifier
    hard = VotingClassifier([('knn10',knn10),('knn5',knn5),('lr',lr),
                            ('dt5',dt5),('dt7',dt7)])
    soft = VotingClassifier([('knn10',knn10),('knn5',knn5),('lr',lr),
                            ('dt5',dt5),('dt7',dt7)],voting='soft')
    
    names = ['hard','soft','knn10','knn5','lr','dt5','dt7']

    for idx, model in enumerate([hard,soft,knn10,lr,dt5,dt7]) :
        model.fit(x_tr,y_tr)
        name = names[idx]
        train_score = model.score(x_tr,y_tr) * 100
        test_score = model.score(x_te,y_te) * 100
        print(f'{name} Train Accuracy:{train_score:.2f}%')
        print(f'{name} Test Accuracy:{test_score:.2f}%')    
        print()

    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(max_depth=5).fit(x_tr,y_tr)
    model.score(x_tr,y_tr), model.score(x_te,y_te)
    
    from sklearn.ensemble import GradientBoostingClassifier
    GBCmodel = GradientBoostingClassifier().fit(x_tr,y_tr)
    GBCmodel.score(x_tr,y_tr), GBCmodel.score(x_te,y_te)
    
    from sklearn.ensemble import StackingClassifier
    estimators = [('rf',RandomForestClassifier()),('gb',GradientBoostingClassifier())]
    model = StackingClassifier(estimators=estimators,final_estimator=LogisticRegression())
    model.fit(x_tr,y_tr).score(x_te,y_te)
    
    sub = pd.DataFrame(GBCmodel.predict_proba(x_te))
    return sub
    
#     knn = KNeighborsClassifier(n_neighbors=10).fit(x_tr,y_tr)
#     knn_pred = knn.predict(x_te)

#     lr = LogisticRegression(max_iter=10000).fit(x_tr,y_tr)
#     lr_pred = lr.predict(x_te)
#     dtc = DecisionTreeClassifier(max_depth=3).fit(x_tr,y_tr)
#     dtc_pred = dtc.predict(x_te)

#     rfc = RandomForestClassifier(max_depth=3).fit(x_tr,y_tr)
#     rfc_pred = rfc.predict(x_te)

#     gbc = GradientBoostingClassifier(max_depth=3).fit(x_tr,y_tr)
#     gbc_pred = rfc.predict(x_te)

#     svm = SVC().fit(x_tr,y_tr)
#     svm_pred = svm.predict(x_te)

#     mlp = MLPClassifier().fit(x_tr,y_tr)
#     mlp_pred = mlp.predict(x_te)
    
#     print('KNN 정확도 : %.2f%%' % (knn.score(x_te,y_te)*100))
#     print('Logistic Regression 정확도 : %.2f%%' % (lr.score(x_te,y_te)*100))
#     print('Decision Tree 정확도 : %.2f%%' % (dtc.score(x_te,y_te)*100))
#     print('Random Forest 정확도 : %.2f%%' % (rfc.score(x_te,y_te)*100))
#     print('Gradient Boosting 정확도 : %.2f%%' % (svm.score(x_te,y_te)*100))
#     print('MLP 정확도 : %.2f%%' % (mlp.score(x_te,y_te)*100))
#     sub = pd.DataFrame(lr.predict_proba(x_te))
#     return sub

In [2]:
problist = {}

In [3]:
IBK기업은행_result = pre(IBK기업은행)

NameError: name 'IBK기업은행' is not defined

In [134]:
흥국생명_result = pre(흥국생명)

KNN 정확도 : 62.63%
Logistic Regression 정확도 : 66.84%
Decision Tree 정확도 : 57.89%
Random Forest 정확도 : 66.32%
Gradient Boosting 정확도 : 63.68%
MLP 정확도 : 63.16%


In [135]:
한국도로공사_result = pre(한국도로공사)

KNN 정확도 : 55.32%
Logistic Regression 정확도 : 62.23%
Decision Tree 정확도 : 62.77%
Random Forest 정확도 : 64.36%
Gradient Boosting 정확도 : 61.17%
MLP 정확도 : 59.04%


In [136]:
현대건설_result = pre(현대건설)

KNN 정확도 : 63.59%
Logistic Regression 정확도 : 60.33%
Decision Tree 정확도 : 61.96%
Random Forest 정확도 : 65.22%
Gradient Boosting 정확도 : 67.39%
MLP 정확도 : 66.30%


In [137]:
GS칼텍스_result = pre(GS칼텍스)

KNN 정확도 : 50.29%
Logistic Regression 정확도 : 59.54%
Decision Tree 정확도 : 59.54%
Random Forest 정확도 : 59.54%
Gradient Boosting 정확도 : 56.65%
MLP 정확도 : 60.12%


In [138]:
KGC인삼공사_result = pre(KGC인삼공사)

KNN 정확도 : 53.22%
Logistic Regression 정확도 : 53.80%
Decision Tree 정확도 : 60.82%
Random Forest 정확도 : 58.48%
Gradient Boosting 정확도 : 54.39%
MLP 정확도 : 56.73%


In [139]:
페퍼저축은행_result = pre(페퍼저축은행)

KNN 정확도 : 90.00%
Logistic Regression 정확도 : 86.67%
Decision Tree 정확도 : 86.67%
Random Forest 정확도 : 90.00%
Gradient Boosting 정확도 : 90.00%
MLP 정확도 : 76.67%


In [265]:
# problist['IBK기업은행']=[IBK기업은행_result[0].mean(),IBK기업은행_result[1].mean()]
# problist['흥국생명']=[흥국생명_result[0].mean(),흥국생명_result[1].mean()]
# problist['한국도로공사']=[한국도로공사_result[0].mean(),한국도로공사_result[1].mean()]
# problist['현대건설']=[현대건설_result[0].mean(),현대건설_result[1].mean()]
# problist['GS칼텍스']=[GS칼텍스_result[0].mean(),GS칼텍스_result[1].mean()]
# problist['KGC인삼공사']=[KGC인삼공사_result[0].mean(),KGC인삼공사_result[1].mean()]
# problist['페퍼저축은행']=[페퍼저축은행_result[0].mean(),페퍼저축은행_result[1].mean()]


In [140]:
problist['IBK기업은행']=IBK기업은행_result[0].mean()
problist['흥국생명']=흥국생명_result[0].mean()
problist['한국도로공사']=한국도로공사_result[0].mean()
problist['현대건설']=현대건설_result[0].mean()
problist['GS칼텍스']=GS칼텍스_result[0].mean()
problist['KGC인삼공사']=KGC인삼공사_result[0].mean()
problist['페퍼저축은행']=페퍼저축은행_result[0].mean()


In [141]:
sort_problist = sorted(problist.items(), key=lambda item: item[1], reverse = True)
sort_problist

[('페퍼저축은행', 0.8928597892156118),
 ('KGC인삼공사', 0.5909106279836929),
 ('현대건설', 0.5478459977487661),
 ('IBK기업은행', 0.5478016299224963),
 ('흥국생명', 0.4525664472793785),
 ('GS칼텍스', 0.4244381443158471),
 ('한국도로공사', 0.38292158224481465)]

In [142]:
teamscore = {}
idx = 1
for i in sort_problist : 
    teamscore[i[0]] = idx
    idx+=1
teamscore

{'페퍼저축은행': 1,
 'KGC인삼공사': 2,
 '현대건설': 3,
 'IBK기업은행': 4,
 '흥국생명': 5,
 'GS칼텍스': 6,
 '한국도로공사': 7}

In [143]:
def wincheck(x,y) :
    if teamscore[x] > teamscore[y] : return 1
    else : return 0

In [144]:
ans = pd.read_csv('Female(2022_34_2)_volleyball_018.csv')
ans

,팀명,경기날짜,결과,이름,포지션,스타팅멤버
0,현대건설,2022-03-01,1,야스민,R,1
1,현대건설,2022-03-01,1,양효진,C,1
2,현대건설,2022-03-01,1,황민경,L,1
3,현대건설,2022-03-01,1,고예림,L,1
4,현대건설,2022-03-01,1,이다현,C,1
...,...,...,...,...,...,...
97,GS칼텍스,2022-03-21,1,강소휘,L,1
98,GS칼텍스,2022-03-21,1,한수지,C,1
99,GS칼텍스,2022-03-21,1,안혜진,S,0
100,GS칼텍스,2022-03-21,1,오지영,Li,0


In [145]:
# 데이터프레임 변경 (날짜, 팀 별 출전 선수 명단과 결과)
ans = ans.groupby(['경기날짜', '팀명'])[['이름', '결과']].agg({'이름' : lambda data : ' '.join(data).split(), '결과' : any})
ans = pd.DataFrame(ans).reset_index()
ans['이름'] = ans['이름'].str[:5]

In [146]:
ans.dtypes

경기날짜    object
팀명      object
이름      object
결과        bool
dtype: object

In [147]:
ans.iloc[:,3][0]

False

In [148]:
ans['결과'] = ans['결과'].apply(lambda x: 1 if x==True else 0)
# import numpy as np
# ans['결과'] = np.where(ans['결과'],1,0)

In [149]:
ans['결과']

0     0
1     1
2     0
3     1
4     1
5     0
6     1
7     0
8     1
9     0
10    1
11    0
Name: 결과, dtype: int64

In [150]:
corr = 0
for i in range(ans.shape[0]) :
    if i%2==0 : continue
    A = ans.loc[i-1,:]['팀명']
    B = ans.loc[i,:]['팀명']
    check = wincheck(A,B)
    if check==ans.iloc[:,3][i] : corr+=1

print(corr/ans.shape[0]*100)

8.333333333333332


### 결론

- 선수 개인의 역량이 합쳐져 팀의 성과를 내는 것이므로, 팀의 종합적인 성공 확률만으로 경기 결과를 예측하기 어렵다

In [119]:
X = data.iloc[:,2:]
y = data.iloc[:,1]

##### 로지스틱 회귀

In [120]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

x_tr, x_te, y_tr, y_te = train_test_split(X,y,
                                         random_state=0,
                                         stratify=y)
Lomodel = LogisticRegression(max_iter=10000).fit(x_tr,y_tr)
Lomodel.score(x_tr,y_tr), Lomodel.score(x_te,y_te)

(0.595795084394433, 0.5950266429840142)

#### KNN

In [121]:
from sklearn.neighbors import KNeighborsRegressor
KNNmodel = KNeighborsRegressor(n_neighbors=3).fit(x_tr,y_tr)
KNNmodel.score(x_tr,y_tr), KNNmodel.score(x_te,y_te)

(0.34765754173665053, -0.18944774005357123)

#### 선형회귀

In [122]:
from sklearn.linear_model import LinearRegression
Limodel = LinearRegression().fit(x_tr,y_tr)
Limodel.score(x_tr,y_tr), Limodel.score(x_te,y_te)

(0.05457844872339257, 0.06052992647887512)

#### 다항회귀

In [77]:
from sklearn.preprocessing import PolynomialFeatures
P = PolynomialFeatures(degree=2)
x_poly = P.fit_transform(X)
x_tr, x_te, y_tr, y_te = train_test_split(x_poly,y,
                                         random_state=0)
print(X.shape,x_poly.shape)

(4503, 16) (4503, 153)


In [78]:
Limodel = LinearRegression().fit(x_tr,y_tr)
Limodel.score(x_tr, y_tr), Limodel.score(x_te,y_te)

(0.11983213834217132, 0.09007641369144026)

#### Decision Tree 

In [82]:
x_tr, x_te, y_tr, y_te = train_test_split(X,y,
                                         random_state=0,
                                         stratify=y)
from sklearn.tree import DecisionTreeRegressor
Demodel = DecisionTreeRegressor().fit(x_tr,y_tr)
Demodel.score(x_tr,y_tr), Demodel.score(x_te,y_te)

(0.8831556465654121, -0.6803043488345759)

#### 모델별 정확도

In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

knn = KNeighborsClassifier(n_neighbors=10).fit(x_tr,y_tr)
knn_pred = knn.predict(x_te)

lr = LogisticRegression(max_iter=10000).fit(x_tr,y_tr)
lr_pred = lr.predict(x_te)


In [95]:
dtc = DecisionTreeClassifier(max_depth=3).fit(x_tr,y_tr)
dtc_pred = dtc.predict(x_te)

rfc = RandomForestClassifier(max_depth=3).fit(x_tr,y_tr)
rfc_pred = rfc.predict(x_te)

gbc = GradientBoostingClassifier(max_depth=3).fit(x_tr,y_tr)
gbc_pred = rfc.predict(x_te)

svm = SVC().fit(x_tr,y_tr)
svm_pred = svm.predict(x_te)

mlp = MLPClassifier().fit(x_tr,y_tr)
mlp_pred = mlp.predict(x_te)

In [96]:
print('KNN 정확도 : %.2f%%' % (knn.score(x_te,y_te)*100))
print('Logistic Regression 정확도 : %.2f%%' % (lr.score(x_te,y_te)*100))
print('Decision Tree 정확도 : %.2f%%' % (dtc.score(x_te,y_te)*100))
print('Random Forest 정확도 : %.2f%%' % (rfc.score(x_te,y_te)*100))
print('Gradient Boosting 정확도 : %.2f%%' % (svm.score(x_te,y_te)*100))
print('MLP 정확도 : %.2f%%' % (mlp.score(x_te,y_te)*100))

KNN 정확도 : 57.55%
Logistic Regression 정확도 : 59.50%
Decision Tree 정확도 : 55.68%
Random Forest 정확도 : 60.66%
Gradient Boosting 정확도 : 57.82%
MLP 정확도 : 62.26%


In [97]:
lr.predict_proba(x_te)

array([[0.90523535, 0.09476465],
       [0.39340982, 0.60659018],
       [0.56557321, 0.43442679],
       [0.61315292, 0.38684708],
       [0.3884212 , 0.6115788 ]])

In [123]:
x_te

,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,스타팅멤버,팀명_GS칼텍스,팀명_IBK기업은행,팀명_KGC인삼공사,팀명_페퍼저축은행,팀명_한국도로공사,팀명_현대건설,팀명_흥국생명,포지션_C,포지션_L,포지션_R,포지션_S
7104,0.00,0.00,0.00,0.00,1,0,0,0,1,0,0,0,0,0,1,0
4558,0.00,86.67,30.77,40.00,1,0,0,0,0,0,0,1,0,1,0,0
7095,0.00,0.00,0.00,0.00,1,0,1,0,0,0,0,0,1,0,0,0
4353,0.00,100.00,9.09,0.00,1,0,0,0,0,0,1,0,1,0,0,0
5500,0.00,100.00,28.57,38.46,1,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,40.00,100.00,20.00,33.33,1,0,0,1,0,0,0,0,0,1,0,0
7487,0.00,0.00,0.00,50.00,1,0,0,0,0,0,0,1,0,0,1,0
1975,40.00,86.67,33.33,23.08,1,0,0,0,0,0,1,0,0,1,0,0
613,25.00,100.00,20.00,0.00,1,0,0,1,0,0,0,0,0,0,1,0


In [102]:
x_te.shape

(1126, 16)

In [103]:
lr.decision_function(x_te)[:20]

array([-2.25679852,  0.43300154, -0.26381232, -0.46058488,  0.4539535 ,
       -1.76260669,  0.35728903,  0.83010275, -0.50664563,  0.36881164,
        0.14523783,  0.05532266, -0.32304555,  0.05197973,  0.29888128,
       -0.05867805, -0.01479256, -0.25341103,  0.36852718, -0.03553009])

In [106]:
model.predict_proba(x_te)[:20]

array([[0.90523535, 0.09476465],
       [0.39340982, 0.60659018],
       [0.56557321, 0.43442679],
       [0.61315292, 0.38684708],
       [0.3884212 , 0.6115788 ],
       [0.85353583, 0.14646417],
       [0.41161597, 0.58838403],
       [0.30362334, 0.69637666],
       [0.6240198 , 0.3759802 ],
       [0.4088282 , 0.5911718 ],
       [0.46375423, 0.53624577],
       [0.48617286, 0.51382714],
       [0.5800663 , 0.4199337 ],
       [0.48700799, 0.51299201],
       [0.42583099, 0.57416901],
       [0.5146653 , 0.4853347 ],
       [0.50369807, 0.49630193],
       [0.56301589, 0.43698411],
       [0.40889695, 0.59110305],
       [0.50888159, 0.49111841]])

In [136]:
model.predict_proba(x_te)

array([0.90523535, 0.09476465])

In [143]:
sub = pd.DataFrame(model.predict_proba(x_te))
sub

,0,1
0,0.905235,0.094765
1,0.393410,0.606590
2,0.565573,0.434427
3,0.613153,0.386847
4,0.388421,0.611579
...,...,...
1121,0.529862,0.470138
1122,0.451776,0.548224
1123,0.446801,0.553199
1124,0.690298,0.309702


In [146]:
finaldata = pd.concat([df_starting,sub],axis=1)
finaldata

,팀명,경기날짜,결과,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,스타팅멤버,포지션,0,1
0,IBK기업은행,2017-10-14,0.0,0.00,83.33,15.00,45.45,1.0,L,0.905235,0.094765
1,IBK기업은행,2017-10-14,0.0,14.29,77.27,0.00,60.00,1.0,L,0.393410,0.606590
2,IBK기업은행,2017-10-14,0.0,0.00,91.67,28.57,50.00,1.0,R,0.565573,0.434427
3,IBK기업은행,2017-10-14,0.0,0.00,83.33,16.67,60.00,1.0,L,0.613153,0.386847
4,IBK기업은행,2017-10-14,0.0,0.00,90.91,3.57,0.00,1.0,C,0.388421,0.611579
...,...,...,...,...,...,...,...,...,...,...,...
8108,KGC인삼공사,2022-02-28,0.0,0.00,0.00,0.00,50.00,1.0,R,NaN,NaN
8109,KGC인삼공사,2022-02-28,0.0,0.00,0.00,0.00,50.00,1.0,L,NaN,NaN
8110,KGC인삼공사,2022-02-28,0.0,0.00,0.00,0.00,33.33,1.0,L,NaN,NaN
8111,KGC인삼공사,2022-02-28,0.0,0.00,0.00,0.00,0.00,1.0,C,NaN,NaN


In [ ]:
df_position = df_starting.groupby('포지션').agg()
df_starting.groupby(['경기날짜','팀명','포지션']).agg('mean')

In [52]:
df_starting.loc[df_starting['팀명']=='IBK기업은행']

,팀명,경기날짜,결과,세트_성공률,디그_성공률,블로킹_성공률,퀵오픈_성공률,스타팅멤버,포지션
0,IBK기업은행,2017-10-14,0,0.00,83.33,15.00,45.45,1,L
1,IBK기업은행,2017-10-14,0,14.29,77.27,0.00,60.00,1,L
2,IBK기업은행,2017-10-14,0,0.00,91.67,28.57,50.00,1,R
3,IBK기업은행,2017-10-14,0,0.00,83.33,16.67,60.00,1,L
4,IBK기업은행,2017-10-14,0,0.00,90.91,3.57,0.00,1,C
...,...,...,...,...,...,...,...,...,...
8072,IBK기업은행,2022-02-26,1,0.00,0.00,0.00,76.47,1,C
8073,IBK기업은행,2022-02-26,1,0.00,0.00,0.00,40.00,1,L
8074,IBK기업은행,2022-02-26,1,0.00,0.00,0.00,0.00,1,C
8075,IBK기업은행,2022-02-26,1,0.00,0.00,0.00,0.00,1,C
